<a href="https://colab.research.google.com/github/Appajisdsrao/FRACTURE-SITE-DETECTION/blob/main/Update.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Step 1: Mount Google Drive
from google.colab import drive

# Mount Google Drive to access dataset
drive.mount('/content/drive')

# Step 2: Verify dataset structure
import os

# Specify the path to your dataset
dataset_dir = '/content/drive/MyDrive/xray_dataset'

# List files in the directory to check the structure
os.listdir(dataset_dir)

# Output should show:
# ['training', 'testing']
# with subdirectories 'fractured' and 'not_fractured' in both 'training' and 'testing'

# Step 3: Import necessary libraries
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras import layers, models

# Step 4: Set up ImageDataGenerators for data augmentation

train_data_dir = '/content/drive/MyDrive/xray_dataset/training'
test_data_dir = '/content/drive/MyDrive/xray_dataset/testing'

# Initialize ImageDataGenerators with augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values to [0,1]
    rotation_range=20,  # Randomly rotate images
    width_shift_range=0.2,  # Randomly shift images horizontally
    height_shift_range=0.2,  # Randomly shift images vertically
    shear_range=0.2,  # Apply shear transformations
    zoom_range=0.2,  # Randomly zoom into images
    horizontal_flip=True,  # Randomly flip images horizontally
    fill_mode='nearest'  # Fill in missing pixels after transformations
)

test_datagen = ImageDataGenerator(rescale=1./255)  # Only rescale for test data

# Set up the data generators for training and testing
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(150, 150),  # Resize images to 150x150
    batch_size=64,  # Larger batch size to accommodate many images
    class_mode='binary'  # Binary classification: fractured vs not fractured
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(150, 150),
    batch_size=64,  # Same batch size for consistency
    class_mode='binary'
)

# Step 5: Build the Convolutional Neural Network (CNN) model

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Binary output (fractured or not)
])

# Step 6: Compile the model
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

# Step 7: Calculate steps per epoch (Total images / Batch size)
steps_per_epoch = train_generator.samples // train_generator.batch_size
validation_steps = test_generator.samples // test_generator.batch_size

# Step 8: Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,  # Total images divided by batch size
    epochs=10,  # Number of epochs
    validation_data=test_generator,
    validation_steps=validation_steps  # Validation steps
)

# Step 9: Save the trained model (optional)
model.save('/content/drive/MyDrive/xray_model.h5')

# Step 10: Plot training and validation accuracy
import matplotlib.pyplot as plt

# Plot training and validation accuracy
plt.plot(history.history['accuracy'], label='Training accuracy')
plt.plot(history.history['val_accuracy'], label='Validation accuracy')
plt.title('Model accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Step 11: Plot training and validation loss
plt.plot(history.history['loss'], label='Training loss')
plt.plot(history.history['val_loss'], label='Validation loss')
plt.title('Model loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/xray_dataset'